In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso, RidgeCV, LassoCV

In [2]:
import pickle5 as pickle
four_years_df = pd.read_pickle("cleaned_df.pkl")

In [3]:
four_years_df = four_years_df[four_years_df['MP']>100]
four_years_df = four_years_df.drop(['POS','Name','YEAR','Season','through_4_years','WITH SHOES','MP'],axis=1)

In [4]:
four_years_df = pd.get_dummies(four_years_df)

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(four_years_df.drop(['BPM','Avg_BPM'],axis=1), four_years_df['Avg_BPM'], test_size = .3, random_state = 1)

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
X_train.shape
Y_train.shape
X_test.shape
Y_test.shape

(200,)

In [7]:
lassocv = LassoCV(alphas=None, n_alphas=200,cv=10, max_iter=100000, normalize=True, random_state=1)
lassocv.fit(X_train, Y_train)

LassoCV(cv=10, max_iter=100000, n_alphas=200, normalize=True, random_state=1)

In [8]:
print(lassocv.alpha_)

0.014557103619576746


In [9]:
lasso_cvalpha = Lasso(alpha=lassocv.alpha_, normalize=True)
lasso_cvalpha.fit(X_train, Y_train)

Lasso(alpha=0.014557103619576746, normalize=True)

In [10]:
lasso_pred = lasso_cvalpha.predict(X_test)

In [11]:
mean_squared_error(Y_test,lasso_pred)

8.49349098707092

In [12]:
pd.Series(lasso_cvalpha.coef_, index=X_train.columns)

NO SHOES      0.000000
REACH         0.000000
WEIGHT        0.000000
WINGSPAN      0.005687
WING DIFF     0.000000
SHUTTLE      -0.000000
3/4 SPRINT   -0.000000
STD VERT     -0.000000
MAX VERT     -0.000000
BENCH        -0.000000
BODY FAT %    0.000000
HAND L       -0.000000
HAND W       -0.000000
after_2010   -0.000000
Pos_C         0.000000
Pos_PF       -0.000000
Pos_PG       -0.000000
Pos_SF        0.000000
Pos_SG       -0.000000
dtype: float64

In [13]:
mod=sm.OLS(Y_train,X_train['WINGSPAN'])
mod.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                Avg_BPM   R-squared (uncentered):                   0.348
Model:                            OLS   Adj. R-squared (uncentered):              0.346
Method:                 Least Squares   F-statistic:                              247.8
Date:                Thu, 14 Oct 2021   Prob (F-statistic):                    4.63e-45
Time:                        13:24:56   Log-Likelihood:                         -1180.8
No. Observations:                 466   AIC:                                      2364.
Df Residuals:                     465   BIC:                                      2368.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
WINGSPAN      -0.0270      0.002    -15.741      0.000      -0.030      -0.024
==============================================================================
Omnibus:                      116.102   Durbin-Watson:                   2.083
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2046.969
Skew:                           0.550   Prob(JB):                         0.00
Kurtosis:                      13.208   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
from pyearth import Earth
spline_model = Earth()
spline_model.fit(X_train,Y_train)
yhat = spline_model.predict(X_test)
print(spline_model.summary())

C:\Users\kylek\anaconda3\lib\site-packages\pyearth\earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
C:\Users\kylek\anaconda3\lib\site-packages\pyearth\earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


Earth()

Earth Model
----------------------------------------
Basis Function     Pruned  Coefficient  
----------------------------------------
(Intercept)        No      -2.92332     
h(WINGSPAN-89.25)  No      5.91425      
h(89.25-WINGSPAN)  Yes     None         
h(WEIGHT-260)      No      -0.138589    
h(260-WEIGHT)      Yes     None         
h(REACH-112)       No      -2.57624     
h(112-REACH)       Yes     None         
SHUTTLE            Yes     None         
Pos_SF             Yes     None         
BENCH              Yes     None         
h(SHUTTLE-3.27)    Yes     None         
h(3.27-SHUTTLE)    No      0.209065     
h(SHUTTLE-2.64)    Yes     None         
h(2.64-SHUTTLE)    Yes     None         
h(BENCH-21)        Yes     None         
h(21-BENCH)        Yes     None         
Pos_PG             Yes     None         
BODY FAT %         Yes     None         
----------------------------------------
MSE: 8.2908, GCV: 8.6965, RSQ: 0.1005, GRSQ: 0.0605
